In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from quantum_tunneling.workflows import run_bound_states, run_field_scan
from quantum_tunneling.fields import apply_field, barrier_top
from quantum_tunneling.viz import plot_turning_points

In [ ]:
potentials = {
    "cusp": {"kind": "cusp", "params": {"V0": 1.0, "alpha": 0.5}},
    "exp_well": {"kind": "exp_well", "params": {"V0": 5.0, "a": 1.0}},
    "soft_barrier": {"kind": "soft_barrier", "params": {"V0": 5.0}},
    "rough": {"kind": "rough", "params": {"V0": 1.0, "k0": 0.6, "levels": 4, "decay": 0.6}},
}
pot_key = "cusp"
state_index = 0
F_grid = np.linspace(0.02, 0.5, 20)
cfg = {
    "potential": potentials[pot_key],
    "grid": {"L": 10.0, "N": 1200},
    "solver": {"k": 6, "hbar": 1.0, "m": 1.0},
}
cfg

In [ ]:
res = run_bound_states(cfg)
scan = run_field_scan(cfg, res, state_index=state_index, F_grid=F_grid)
records = scan['records']
F_vals = [r['F'] for r in records]
lnT = [np.log(r['T']) if 'T' in r else np.nan for r in records]
plt.figure()
plt.plot(F_vals, lnT, marker='o')
plt.xlabel('F')
plt.ylabel('ln T_WKB')
plt.title('Transmission vs Field')
plt.grid(True, alpha=0.3)
plt.show()
[(r['F'], r['status']) for r in records]

In [ ]:
# Visualize tilted barriers at a few F values.
sample_F = [F_vals[0], F_vals[len(F_vals)//2], F_vals[-1]]
x = res['x']
E = res['E'][state_index]
for F in sample_F:
    Vtilt = apply_field(res['Vx'], x, F)
    plt.figure()
    plt.plot(x, Vtilt, label=f'V-Fx (F={F:.2f})')
    plt.axhline(E, color='k', linestyle='--', label='E')
    plt.xlabel('x')
    plt.ylabel('Energy')
    plt.legend()
    plt.title('Tilted barrier snapshot')
    plt.tight_layout()
    plt.show()

## Notes
- `status` flags where barrier disappears (over-the-barrier).
- Compare slopes of ln T vs F between potentials; cusps should deviate from smooth barriers.
- Swap `state_index` to contrast ground vs excited-state tunneling.